# Botnet IP-Location Data

In [ ]:
from sqlalchemy import create_engine
from declarative import DB_DIALECT, DB_DIR, DB_NAME
from declarative import SrcIPLocation, LoginSuccess, LoginFailed, CommandInput
import pandas as pd
from collections import Counter


## Queries:

In [ ]:
ECHO = False

pd.set_option('display.max_rows', None)

engine = create_engine(DB_DIALECT + DB_DIR + DB_NAME, echo=ECHO)

loc = pd.read_sql_table(SrcIPLocation.__tablename__, engine)[['id', 'src_ip', 'country_name', 'state_name', 'city_name']]
success = pd.read_sql_table(LoginSuccess.__tablename__, engine)[['timestamp', 'src_ip']]
failed = pd.read_sql_table(LoginFailed.__tablename__, engine)[['timestamp', 'src_ip', 'username', 'password']]
success_loc = pd.merge(loc, success, on='src_ip')
failed_loc = pd.merge(loc, failed, on='src_ip')
inputs = pd.read_sql_table(CommandInput.__tablename__, engine)


## Total Unique IP Count:

In [ ]:
loc.count()['src_ip']

## Most Common Countries of Origin:

In [ ]:
country_count = Counter(loc['country_name'])
country_ips = pd.DataFrame(country_count.most_common(10), columns=['country_name', 'unique_ips'])
country_ips


## Successful Login Count:

In [ ]:
success_loc.count()['src_ip']

## Successful Unique IP Count:

In [ ]:
ips = success['src_ip']
len(set(ips))

## Total Unique IP Location Data:

In [ ]:
loc.sort_values(by='country_name')

## Successful Login IP Data:

In [ ]:
success_loc.sort_values(by='timestamp', ascending=False)[['id', 'timestamp', 'src_ip', 'country_name', 
                                                          'state_name', 'city_name']]

## Most Recent Login Failures:

In [ ]:
failed_loc.sort_values(by='timestamp', ascending=False)[['id', 'timestamp', 'src_ip', 'country_name', 
                                                          'state_name', 'city_name', 'username', 'password']].head(50)

## Most Common Failed Login Attempts:

In [ ]:
df_failed_logins = failed[['username', 'password']]
l_failed_logins = [(x[0], x[1]) for x in df_failed_logins.values]
c_failed_logins = Counter(l_failed_logins).most_common(20)

pd.DataFrame([[x[0][0], x[0][1], x[1]] for x in c_failed_logins], columns=['username', 'password', 'count'])
    